In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2, p3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 70, bias= True),
            nn.ReLU(),
            nn.Linear(70, 70, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(70, 65, bias = True),
            nn.ReLU(),
            nn.Linear(65, 60, bias= True),
            nn.ReLU(),
            nn.Linear(60, 60, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(60, 55, bias= True),
            nn.ReLU(),
            nn.Linear(55, 50, bias= True),
            nn.ReLU(),
            nn.Linear(50, 45, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p3),
            nn.Linear(45, 40, bias= True),
            nn.ReLU(),
            nn.Linear(40, 35, bias = True),
            nn.ReLU(),
            nn.Linear(35, 25, bias = True),
            nn.ReLU(),
            nn.Linear(25, 20, bias = True),
            nn.ReLU(),
            nn.Linear(20, 15, bias = True),
            nn.ReLU(),
            nn.Linear(15, 10, bias = True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True)
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=100, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 100)

In [5]:
model = Network(0.3, 0.4, 0.5)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=70, bias=True)
    (1): ReLU()
    (2): Linear(in_features=70, out_features=70, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=70, out_features=65, bias=True)
    (6): ReLU()
    (7): Linear(in_features=65, out_features=60, bias=True)
    (8): ReLU()
    (9): Linear(in_features=60, out_features=60, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=60, out_features=55, bias=True)
    (13): ReLU()
    (14): Linear(in_features=55, out_features=50, bias=True)
    (15): ReLU()
    (16): Linear(in_features=50, out_features=45, bias=True)
    (17): ReLU()
    (18): Dropout(p=0.5, inplace=False)
    (19): Linear(in_features=45, out_features=40, bias=True)
    (20): ReLU()
    (21): Linear(in_features=40, out_features=35, bias=True)
    (22): ReLU()
    (23): Linear(in_features=35, out_features=25, bias=True)
    (24): ReLU

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay= 0.00001)

In [7]:
for epoch in range(500):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 3326.611416876316
loss - 3311.2289603352547
loss - 3309.066664993763
loss - 3308.5516813397408
loss - 3308.204633295536
loss - 3307.7973853349686
loss - 3307.597231864929
loss - 3307.542789340019
loss - 3306.8666843771935
loss - 3304.089122056961
loss - 3299.6179278492928
loss - 3296.9908217191696
loss - 3295.5270941257477
loss - 3294.0623101592064
loss - 3292.8525152802467
loss - 3292.023191511631
loss - 3290.1053526997566
loss - 3289.008346438408
loss - 3287.9190333485603
loss - 3286.9637102484703
loss - 3285.536504626274
loss - 3284.2332589030266
loss - 3282.9880378246307
loss - 3282.4989913105965
loss - 3281.572328686714
loss - 3281.0880573391914
loss - 3280.0877279639244
loss - 3279.949222743511
loss - 3278.475438773632
loss - 3277.768176674843
loss - 3278.1237681508064
loss - 3277.0308917164803
loss - 3276.6535474061966
loss - 3275.9127303361893
loss - 3275.038052916527
loss - 3274.685696899891
loss - 3273.794310927391
loss - 3273.972232878208
loss - 3272.8830310106277
los

In [9]:
for epoch in range(1):
    model.train()
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        for x in model.parameters():
            print(x.grad)
        break
        

tensor([[ 2.3218e-06,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          6.9653e-07,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-1.9163e-05, -5.1164e-06, -4.7907e-09,  ...,  0.0000e+00,
         -3.3535e-08, -4.7907e-09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.7666e-01, -1.5888e-03,  0.0000e+00,  ...,  0.0000e+00,
          3.1678e-04,  0.0000e+00]], device='cuda:0')
tensor([ 2.3218e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00, -2.6886e-04,
         6.9709e-07,  1.5613e-04,  2.2443e-04, -2.7074e-04, -2.0523e-05,
        -1.5421e-04,  4.0631e-04,  3.5890e-08,  3.9449e-08,  0.0000e+00,
         0.0000e+00,  6.9702e-07,  0.0000e+00, -1.4075e-07,  0.0000e+00,
         5.7531e-05, -4.2852e-05, -3.3382e-07, -6.2504e-04,

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.627637848876787
